In [50]:
import numpy as np

In [51]:
# Policy parameterization: Softmax
# State value parameterization: Linear
# State feature vector representation: Fourier (as in HW2)

In [52]:
M = 3 # dimensionality of the fourier transform [2, 10]
# mean_policy_vector = np.ones(2*M+1)*0.05 #some non-zero vector

policy_params = np.ones((2*M+1,2))*0.01
value_params = np.ones(2*M+1)*0.01

softmax_sigma = 0.1
# policy_params = np.zeros((np.power(M+1,2),2))
# value_params = np.zeros(np.power(M+1,2))

policy_step_size = 1e-7 #2e-9
valueFunction_step_size = 1e-7
gamma = 1

In [53]:
def phi_s_cosine(given_x, given_v, cosineFlag):
    normal_x = normalized_x(given_x, cosineFlag)
    normal_v = normalized_v(given_v, cosineFlag)
    feature_phi = np.zeros(2*M+1)
#     feature_phi = np.zeros(np.power(M+1,2))
    feature_phi[0] = 1
    if cosineFlag:
        for i in range(1, M+1):
            feature_phi[i] = np.cos(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.cos(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
    
    else:
        for i in range(1, M+1):
            feature_phi[i] = np.sin(i*np.pi*normal_x)
        
        for i in range(1, M+1):
            feature_phi[M+i] = np.sin(i*np.pi*normal_v)

#         for i in range(0,M+1):
#             for j in range(0,M+1):
#                 if i==0 or j==0:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))
#                 else:
#                     feature_phi[(M+1)*i + j] = np.cos(np.pi*(i*normal_x + j*normal_v))

    return feature_phi

In [54]:
def normalized_x(given_x, cosineFlag):
    if cosineFlag:
        return (given_x+1.2)/(0.5+1.2)
    else:
        return ((given_x+1.2)/(0.5+1.2))*2 - 1
def normalized_v(given_v, cosineFlag):
    if cosineFlag:
        return (given_v+0.7)/(0.7+0.7)
    else:
        return ((given_v+0.7)/(0.7+0.7))*2 - 1

In [55]:
def runEpisode(policy_params, gamma):
    
    episode_trace = []
    
    total_reward = 0
    x_0 = np.random.uniform(-0.6,-0.4,1)[0]
    begin_state = (x_0,0)
    
    S_t = begin_state
    S_t_plus_1 = begin_state
    for t in range(0, 1000):
        
        state_feature_vector = phi_s_cosine(S_t[0], S_t[1], True)
        state_feature_policy_param_product = np.dot(policy_params.T, state_feature_vector)
        exp_terms = np.exp(softmax_sigma * state_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
#         A_t = np.random.choice([-1,1], p=list(pi_values))
        
        max_action_idx = np.argmax(pi_values)
        if max_action_idx==1:
            A_t = 1
        else:
            A_t = -1
        
#         threshold = np.dot(state_feature_vector, given_theta)
        
#         print(threshold)
        
#         if threshold<=0:
#             A_t = -1
#         else:
#             A_t = 1
        
        v_t_plus_1 = S_t[1] + 0.001*A_t - 0.0025*np.cos(3*S_t[0])
        x_t_plus_1 = S_t[0] + v_t_plus_1
        
        
        
        if x_t_plus_1 < -1.2:
            x_t_plus_1 = -1.2
            v_t_plus_1 = 0
        elif x_t_plus_1 > 0.5:
            x_t_plus_1 = 0.5
            v_t_plus_1 = 0
        
        if v_t_plus_1 < -0.7:
            v_t_plus_1 = -0.7
        elif v_t_plus_1 > 0.7:
            v_t_plus_1 = 0.7
        
        S_t_plus_1 = (x_t_plus_1, v_t_plus_1)
        
        if x_t_plus_1 == 0.5:
            print("Terminated")
            episode_trace.append((S_t, A_t, 0))
            break
        else:
            episode_trace.append((S_t, A_t, -1))
            total_reward += -1
#         print("After ",total_reward)
        

        S_t = S_t_plus_1 #Incrementing time step
    return episode_trace

In [56]:
#With baseline

curr_iter = 0
while(curr_iter<=5000):
    curr_iter += 1
    
    episode_trace = runEpisode(policy_params, gamma)
    print("\n EPISODE LENGTH: ",len(episode_trace), "CURR ITER: ", curr_iter)
    policy_params_temp = policy_params.copy()
    for a_step in range(len(episode_trace)):
        G = 0
        for idx in range(a_step, len(episode_trace)):
            G += episode_trace[idx][2]
        
        state_t = episode_trace[a_step][0]
        state_t_feature_vector = phi_s_cosine(state_t[0], state_t[1], True)

        v_hat_s = np.dot(state_t_feature_vector, value_params)
        delta = G - v_hat_s
        
        value_params += valueFunction_step_size*delta*state_t_feature_vector
        

        state_t_feature_policy_param_product = np.dot(policy_params.T, state_t_feature_vector)
        
        
        exp_terms = np.exp(softmax_sigma*state_t_feature_policy_param_product)
        pi_values = exp_terms/np.sum(exp_terms)
        
#         print("Pi values: ",pi_values)
#         print("G: ",G,". Delta: ",delta)
        if episode_trace[a_step][1] == -1:
            policy_params[:,0] += policy_step_size*delta*(1-pi_values[0])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(-pi_values[0])*state_t_feature_vector
        else:
            policy_params[:,0] += policy_step_size*delta*(-pi_values[1])*state_t_feature_vector
            policy_params[:,1] += policy_step_size*delta*(1-pi_values[1])*state_t_feature_vector
    
    max_diff = np.max(np.abs(policy_params_temp - policy_params))
    print(policy_step_size," Max diff: ",max_diff)
    if max_diff<policy_step_size*1e2 or len(episode_trace)<110:# 0.001 works with 1e-6 policy_step
        break


 EPISODE LENGTH:  1000 CURR ITER:  1
1e-07  Max diff:  0.02512379783833669

 EPISODE LENGTH:  1000 CURR ITER:  2
1e-07  Max diff:  0.02508200373539345

 EPISODE LENGTH:  1000 CURR ITER:  3
1e-07  Max diff:  0.00948888788985615

 EPISODE LENGTH:  1000 CURR ITER:  4
1e-07  Max diff:  0.020463306143821698

 EPISODE LENGTH:  1000 CURR ITER:  5
1e-07  Max diff:  0.025016649726914957

 EPISODE LENGTH:  1000 CURR ITER:  6
1e-07  Max diff:  0.013179325659668808

 EPISODE LENGTH:  1000 CURR ITER:  7
1e-07  Max diff:  0.01836255487720563

 EPISODE LENGTH:  1000 CURR ITER:  8
1e-07  Max diff:  0.02501570142351381

 EPISODE LENGTH:  1000 CURR ITER:  9
1e-07  Max diff:  0.012616762160871915

 EPISODE LENGTH:  1000 CURR ITER:  10
1e-07  Max diff:  0.00501730330641928

 EPISODE LENGTH:  1000 CURR ITER:  11
1e-07  Max diff:  0.003298226971808328

 EPISODE LENGTH:  1000 CURR ITER:  12
1e-07  Max diff:  0.006085698579635715

 EPISODE LENGTH:  1000 CURR ITER:  13
1e-07  Max diff:  0.0024015376493915913


1e-07  Max diff:  0.014576298425117915

 EPISODE LENGTH:  1000 CURR ITER:  107
1e-07  Max diff:  0.014133502987171646

 EPISODE LENGTH:  1000 CURR ITER:  108
1e-07  Max diff:  0.018989947220995028

 EPISODE LENGTH:  1000 CURR ITER:  109
1e-07  Max diff:  0.024276894315446277

 EPISODE LENGTH:  1000 CURR ITER:  110
1e-07  Max diff:  0.014801064095478694

 EPISODE LENGTH:  1000 CURR ITER:  111
1e-07  Max diff:  0.007761287637440169

 EPISODE LENGTH:  1000 CURR ITER:  112
1e-07  Max diff:  0.009581767483404521

 EPISODE LENGTH:  1000 CURR ITER:  113
1e-07  Max diff:  0.012564485225184979

 EPISODE LENGTH:  1000 CURR ITER:  114
1e-07  Max diff:  0.0243195645082991

 EPISODE LENGTH:  1000 CURR ITER:  115
1e-07  Max diff:  0.024221626307656882

 EPISODE LENGTH:  1000 CURR ITER:  116
1e-07  Max diff:  0.01230121682041022

 EPISODE LENGTH:  1000 CURR ITER:  117
1e-07  Max diff:  0.019044925883966946

 EPISODE LENGTH:  1000 CURR ITER:  118
1e-07  Max diff:  0.01530025841180808

 EPISODE LENGTH:

1e-07  Max diff:  0.007712653031388694

 EPISODE LENGTH:  1000 CURR ITER:  212
1e-07  Max diff:  0.009526373426957065

 EPISODE LENGTH:  1000 CURR ITER:  213
1e-07  Max diff:  0.023659439753861363

 EPISODE LENGTH:  1000 CURR ITER:  214
1e-07  Max diff:  0.02353703142282327

 EPISODE LENGTH:  1000 CURR ITER:  215
1e-07  Max diff:  0.012184764513531864

 EPISODE LENGTH:  1000 CURR ITER:  216
1e-07  Max diff:  0.011580867473048893

 EPISODE LENGTH:  1000 CURR ITER:  217
1e-07  Max diff:  0.023679368604644164

 EPISODE LENGTH:  1000 CURR ITER:  218
1e-07  Max diff:  0.023571004306190114

 EPISODE LENGTH:  1000 CURR ITER:  219
1e-07  Max diff:  0.012801587636372251

 EPISODE LENGTH:  1000 CURR ITER:  220
1e-07  Max diff:  0.011564185780860207
Terminated

 EPISODE LENGTH:  646 CURR ITER:  221
1e-07  Max diff:  0.00601911728863049

 EPISODE LENGTH:  1000 CURR ITER:  222
1e-07  Max diff:  0.013131064135346837

 EPISODE LENGTH:  1000 CURR ITER:  223
1e-07  Max diff:  0.023635995297968636

 EPI


 EPISODE LENGTH:  1000 CURR ITER:  316
1e-07  Max diff:  0.02295850532943578

 EPISODE LENGTH:  1000 CURR ITER:  317
1e-07  Max diff:  0.012633931225971597

 EPISODE LENGTH:  1000 CURR ITER:  318
1e-07  Max diff:  0.006208920947748976

 EPISODE LENGTH:  1000 CURR ITER:  319
1e-07  Max diff:  0.006558020235683998

 EPISODE LENGTH:  1000 CURR ITER:  320
1e-07  Max diff:  0.008490660186073102

 EPISODE LENGTH:  1000 CURR ITER:  321
1e-07  Max diff:  0.022843915679123063

 EPISODE LENGTH:  1000 CURR ITER:  322
1e-07  Max diff:  0.02306208811415547

 EPISODE LENGTH:  1000 CURR ITER:  323
1e-07  Max diff:  0.01015589188705961

 EPISODE LENGTH:  1000 CURR ITER:  324
1e-07  Max diff:  0.013509424951302697

 EPISODE LENGTH:  1000 CURR ITER:  325
1e-07  Max diff:  0.02294593853417811

 EPISODE LENGTH:  1000 CURR ITER:  326
1e-07  Max diff:  0.02299253422227443

 EPISODE LENGTH:  1000 CURR ITER:  327
1e-07  Max diff:  0.014105235932860873

 EPISODE LENGTH:  1000 CURR ITER:  328
1e-07  Max diff: 

Terminated

 EPISODE LENGTH:  843 CURR ITER:  420
1e-07  Max diff:  0.009630458131651642

 EPISODE LENGTH:  1000 CURR ITER:  421
1e-07  Max diff:  0.022181747778492078

 EPISODE LENGTH:  1000 CURR ITER:  422
1e-07  Max diff:  0.022452944866491978

 EPISODE LENGTH:  1000 CURR ITER:  423
1e-07  Max diff:  0.007057721547820438

 EPISODE LENGTH:  1000 CURR ITER:  424
1e-07  Max diff:  0.008529540181048578
Terminated

 EPISODE LENGTH:  642 CURR ITER:  425
1e-07  Max diff:  0.0032092676137407163
Terminated

 EPISODE LENGTH:  236 CURR ITER:  426
1e-07  Max diff:  0.0002611323309273314
Terminated

 EPISODE LENGTH:  202 CURR ITER:  427
1e-07  Max diff:  0.0001280755551149905
Terminated

 EPISODE LENGTH:  147 CURR ITER:  428
1e-07  Max diff:  6.12391394041456e-05
Terminated

 EPISODE LENGTH:  165 CURR ITER:  429
1e-07  Max diff:  6.875529732660451e-05
Terminated

 EPISODE LENGTH:  143 CURR ITER:  430
1e-07  Max diff:  5.2027177724322735e-05
Terminated

 EPISODE LENGTH:  149 CURR ITER:  431
1e-07